# Load LonBoard Overture

In [4]:
%pip install lonboard

  Using cached lonboard-0.10.4-py3-none-any.whl.metadata (5.1 kB)
  Using cached anywidget-0.9.18-py3-none-any.whl.metadata (8.9 kB)
Using cached lonboard-0.10.4-py3-none-any.whl (884 kB)
Using cached anywidget-0.9.18-py3-none-any.whl (220 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 7.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 8.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 8.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 5.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.9/853.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [lonboard]8/9 [lonboard]]]te]gets]
Note: you may need to restart the kernel to use updated packages.


In [5]:
import overturemaps
from lonboard import Map, PathLayer

In [6]:
bbox = 41.97642361741863, -87.91712524447735, 41.65606674790702, -87.5403192813114 # Chicago Metro Area

Road Data

In [7]:
table = overturemaps.record_batch_reader("segment", bbox).read_all()
table.shape
table.schema

id: string
geometry: binary
  -- field metadata --
  ARROW:extension:name: 'geoarrow.wkb'
bbox: struct<xmin: float, xmax: float, ymin: float, ymax: float> not null
  child 0, xmin: float
  child 1, xmax: float
  child 2, ymin: float
  child 3, ymax: float
theme: string
version: int32
sources: list<element: struct<property: string, dataset: string, record_id: string, update_time: string, confidence: double>>
  child 0, element: struct<property: string, dataset: string, record_id: string, update_time: string, confidence: double>
      child 0, property: string
      child 1, dataset: string
      child 2, record_id: string
      child 3, update_time: string
      child 4, confidence: double
subtype: string
class: string
names: struct<primary: string, common: map<string, string ('common')>, rules: list<element: struct<variant: string, language: string, value: string, between: list<element: double>, side: string>>>
  child 0, primary: string
  child 1, common: map<string, string ('common')

In [ ]:
#!/usr/bin/env python3
"""
download_businesses_direct.py

Usage:
    python download_businesses_direct.py \
      --bbox="-122.42,37.77,-122.41,37.78" \
      --out businesses.geojson
"""



def download_pois(bbox, output_parquet):
    """
    Programmatically invoke the Overture Maps downloader.
    """
    # note: download() signature mirrors the CLI flags
    download(
        bbox=",".join(map(str, bbox)),
        f="parquet",
        t="place",
        o=output_parquet,
    )
    print(f"▶️ Downloaded POIs to {output_parquet}")


def filter_businesses(input_parquet, output_geojson):
    """
    Reads the POI Parquet, filters by keywords, writes GeoJSON.
    """
    gdf = gpd.read_parquet(input_parquet)
    keywords = [
        "shop", "store", "market", "restaurant", "cafe", "bar",
        "office", "service", "retail"
    ]
    mask = gdf["category"].apply(
        lambda cats: any(kw in cats.lower() for kw in keywords)
    )
    biz = gdf[mask]
    biz.to_file(output_geojson, driver="GeoJSON")
    print(f"✅ Exported {len(biz)} businesses to {output_geojson}")


if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description="Download & extract businesses via the overturemaps Python API"
    )
    parser.add_argument(
        "--bbox", required=True,
        help="min_lon,min_lat,max_lon,max_lat"
    )
    parser.add_argument(
        "--out", default="businesses.geojson",
        help="GeoJSON to save filtered businesses"
    )
    args = parser.parse_args()

    bbox = tuple(map(float, args.bbox.split(",")))
    tmp_parquet = "pois.parquet"

    download_pois(bbox, tmp_parquet)
    filter_businesses(tmp_parquet, args.out)
